In [14]:
import pandas as pd 
import numpy as np 
from scipy import sparse
from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict
from itertools import combinations #genera combinaciones 

In [15]:
dfOrder = pd.read_csv('order_products__train.csv')
dfOrder

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [16]:
dfProduct = pd.read_csv('products.csv')
dfProduct

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [17]:
#Etapa 1, tarea 2
#Identificacion de los N productos mas frecuentes
N = 1000 #Este numero puede variar, solo considerar mientras mas alto, mas es el coste computacional 

top_prod = dfOrder['product_id'].value_counts().nlargest(N).index

In [18]:
#Filtro para incluir los productos frecuentes
dfOrder_fill = dfOrder[dfOrder['product_id'].isin(top_prod)]

#Creamos matriz binaria
order_matrix = pd.crosstab(dfOrder_fill['order_id'], dfOrder_fill['product_id'],  dropna=False)


In [19]:
#Conversion binario 1 para presencia, 0 para ausencia
dfBi = (order_matrix > 0).astype(int)
dfBi.head()

product_id,34,45,196,260,311,329,365,432,581,651,...,49175,49191,49235,49247,49383,49520,49533,49605,49610,49683
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
36,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#Esto es para verificar si me esta arrojando los datos correctos(fin informativo)
print(f"Forma del DataFrame: {dfBi.shape}")
print(f"Valores únicos: {np.unique(dfBi.values)}")
print(f"Número de True: {(dfBi == True).values.sum()}")
print(f"Número de False: {(dfBi == False).values.sum()}")
print(f"Porcentaje de True: {(dfBi == True).values.sum() / dfBi.size * 100:.2f}%")

Forma del DataFrame: (120216, 1000)
Valores únicos: [0 1]
Número de True: 728595
Número de False: 119487405
Porcentaje de True: 0.61%


In [21]:
#Etapa 1, tarea 2 continuacion
#Umbral de soporte minimo (es de ejemplo modificable si es necesario)
min_support = 0.01

#Calculo de soporte de cada item
item_support = dfBi.sum() / len(dfBi) #Esto es la frecuencia de aparicion del producto en el total de transacciones

#aca volvemos a filtrar para ver si cumplem el soporte minimo colocado
frequent_item = item_support[item_support >= min_support]

In [22]:
#ordene por soporte de forma descendente
frequent_item = frequent_item.sort_values(ascending=False)

#Visualizacion de respuesta y verificacion de que el codigo funciona
print(f"1-itemsets frecuentes encontrados: {len(frequent_item)}")
print("\nTop 10 productos más frecuentes:")
print(frequent_item.head(10))

1-itemsets frecuentes encontrados: 118

Top 10 productos más frecuentes:
product_id
24852    0.155770
13176    0.128768
21137    0.090620
21903    0.081387
47626    0.067670
47766    0.061631
47209    0.060666
16797    0.054019
26209    0.050185
27966    0.046134
dtype: float64


In [23]:
# Crear un DataFrame con los 1-itemsets frecuentes para uso posterior
frequentDf = pd.DataFrame({'itemset': [[product] for product in frequent_item.index], 'support': frequent_item.values})

print("DataFrame de 1-itemsets frecuentes:")
print(frequent_item)

#Guardado los indices de los items frecuentes (esto podemos usarlo para generar 2-itemsets)
frequent_items = frequent_item.index.tolist()


DataFrame de 1-itemsets frecuentes:
product_id
24852    0.155770
13176    0.128768
21137    0.090620
21903    0.081387
47626    0.067670
           ...   
41844    0.010132
260      0.010107
24489    0.010090
41787    0.010040
38689    0.010032
Length: 118, dtype: float64


In [24]:
#Etapa 1, tarea 3
def estructuracion(dfOrder, top_n= 1000):
    #Obtengo los productos mas frecuentes
    top_prod= dfOrder['product_id'].value_counts().nlargest(top_n).index
    #filtro para incluir dichos productos
    dfOrder_fill = dfOrder[dfOrder['product_id'].isin(top_prod)]
    #Aqui creo el dicci
    transactions_dict = defaultdict(set)
    #Hago una funcion para agrupar los productos por orden
    for _, row in dfOrder_fill.iterrows():
        transactions_dict[row['order_id']].add(row['product_id'])
    return dict(transactions_dict)
transactions_dict = estructuracion(dfOrder, top_n=N)

In [25]:
#Etapa 1, tarea 4

# Generar candidatos 2-itemsets (generar candidatos k+1)
frequent_item2 = list(combinations(frequent_items, 2))
print(f"Número de 2-itemsets candidatos: {len(frequent_item2)}")
print("Ejemplo de 2-itemsets candidatos:")
print(frequent_item2[:5])
      
# Esta función calcula el soporte para los 2-itemsets
def calculate_support(itemset, transactions):
    """
    Calcula el soporte de un itemset en las transacciones.

    Parámetros:
    - itemset (tuple or set): El conjunto de ítems cuyo soporte se desea calcular.
    - transactions (iterable of sets): Las transacciones, cada una como un conjunto de ítems.

    Retorna:
    - soporte (float): La proporción de transacciones que contienen el itemset.
    """
    # Contar cuántas transacciones contienen el itemset
    count = sum(1 for transaction in transactions if set(itemset).issubset(transaction))
    
    # Dividir entre el total de transacciones para obtener el soporte
    return count / len(transactions)

frequent_item2_support = {}
for itemset in frequent_item2:
    support = calculate_support(itemset, transactions_dict.values())  # Calcula el soporte del itemset
    if support >= min_support:  # Filtrar solo los que cumplen con el soporte mínimo
        frequent_item2_support[itemset] = support

print(f"Número de 2-itemsets frecuentes: {len(frequent_item2_support)}")
print("Ejemplo de 2-itemsets frecuentes:")
print(list(frequent_item2_support.items())[:5])


Número de 2-itemsets candidatos: 6903
Ejemplo de 2-itemsets candidatos:
[(24852, 13176), (24852, 21137), (24852, 21903), (24852, 47626), (24852, 47766)]
Número de 2-itemsets frecuentes: 22
Ejemplo de 2-itemsets frecuentes:
[((24852, 21137), 0.018084115259200107), ((24852, 21903), 0.016636720569641314), ((24852, 47626), 0.017951021494642977), ((24852, 47766), 0.018433486391162573), ((24852, 16797), 0.016204165834830638)]


In [26]:

# Crear DataFrame de 2-itemsets frecuentes
frequent_item2_df = pd.DataFrame({
    'itemset': list(frequent_item2_support.keys()),
    'support': list(frequent_item2_support.values())
})

# Imprimir resultados
print("DataFrame de 2-itemsets frecuentes:")
print(frequent_item2_df.head())


#NOTA: la implementación es eficiente para la generación de candidatos, pero  a medida que aumenTA k ,el número de combinaciones crecerá exponencialmente
#esto podría requerir optimizaciones adicioanles la sugerencia es usar estructuras como tablas hash para filtrar candidatos

DataFrame de 2-itemsets frecuentes:
          itemset   support
0  (24852, 21137)  0.018084
1  (24852, 21903)  0.016637
2  (24852, 47626)  0.017951
3  (24852, 47766)  0.018433
4  (24852, 16797)  0.016204


In [27]:
print(f"Cantidad de 2-itemsets frecuentes: {len(frequent_item2_support)}")
print(f"Ejemplo de 2-itemsets frecuentes:")
print(list(frequent_item2_support.items())[:10])  # Muestra los primeros 10


Cantidad de 2-itemsets frecuentes: 22
Ejemplo de 2-itemsets frecuentes:
[((24852, 21137), 0.018084115259200107), ((24852, 21903), 0.016636720569641314), ((24852, 47626), 0.017951021494642977), ((24852, 47766), 0.018433486391162573), ((24852, 16797), 0.016204165834830638), ((24852, 26209), 0.011071737539096294), ((24852, 45066), 0.010239901510614227), ((24852, 28204), 0.010065215944632994), ((13176, 21137), 0.025570639515538698), ((13176, 21903), 0.01859985359685899)]


In [28]:
# Convertir las claves de frequent_item2_support a tuplas
frequent_item2_support = {tuple(key): value for key, value in frequent_item2_support.items()}


In [29]:
#Extraer los elementos únicos de los 2-itemsets frecuentes
frequent_items_3 = set()
for itemset in frequent_item2_support.keys():  # Aquí usamos los itemsets frecuentes de 2
    frequent_items_3.update(itemset)

# Crear combinaciones de 3 elementos
candidates_3itemsets = list(combinations(frequent_items_3, 3))
print(f"Número de candidatos 3-itemsets generados: {len(candidates_3itemsets)}")
print("Ejemplo de candidatos 3-itemsets generados:")
print(candidates_3itemsets[:5])


Número de candidatos 3-itemsets generados: 364
Ejemplo de candidatos 3-itemsets generados:
[(26209, 47209, 47626), (26209, 47209, 45066), (26209, 47209, 28204), (26209, 47209, 39275), (26209, 47209, 21903)]


In [30]:
#Extraer los elementos únicos de los 2-itemsets frecuentes
frequent_items_3 = set()
for itemset in frequent_item2_support.keys():  # Aquí usamos los itemsets frecuentes de 2
    frequent_items_3.update(itemset)

# Crear combinaciones de 3 elementos
candidates_3itemsets = list(combinations(frequent_items_3, 3))
print(f"Número de candidatos 3-itemsets generados: {len(candidates_3itemsets)}")
print("Ejemplo de candidatos 3-itemsets generados:")
print(candidates_3itemsets[:5])


Número de candidatos 3-itemsets generados: 364
Ejemplo de candidatos 3-itemsets generados:
[(26209, 47209, 47626), (26209, 47209, 45066), (26209, 47209, 28204), (26209, 47209, 39275), (26209, 47209, 21903)]


In [31]:
for candidate in candidates_3itemsets:
    subsets = list(combinations(candidate, len(candidate) - 1))
    print(f"Subconjuntos de {candidate}: {subsets}")
    print(f"Frecuencia de subconjuntos: {[subset in frequent_item2_support for subset in subsets]}")


Subconjuntos de (26209, 47209, 47626): [(26209, 47209), (26209, 47626), (47209, 47626)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 45066): [(26209, 47209), (26209, 45066), (47209, 45066)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 28204): [(26209, 47209), (26209, 28204), (47209, 28204)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 39275): [(26209, 47209), (26209, 39275), (47209, 39275)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 21903): [(26209, 47209), (26209, 21903), (47209, 21903)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 21137): [(26209, 47209), (26209, 21137), (47209, 21137)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 24852): [(26209, 47209), (26209, 24852), (47209, 24852)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 477

In [32]:
# Comparar subconjuntos faltantes
for subset in subsets:
    if subset not in frequent_item2_support:
        print(f"Subconjunto no encontrado: {subset}")


Subconjunto no encontrado: (13176, 16797)
Subconjunto no encontrado: (16797, 27966)


In [33]:
# Revisar el soporte de subconjuntos específicos
missing_subsets = [(13176, 16797), (16797, 27966)]  # Subconjuntos que no se encuentran
for subset in missing_subsets:
    print(f"Subconjunto: {subset}, Soporte: {frequent_item2_support.get(subset, 'No encontrado')}")


Subconjunto: (13176, 16797), Soporte: No encontrado
Subconjunto: (16797, 27966), Soporte: No encontrado


In [34]:
def apriori_prune(candidates, frequent_itemsets_k):
    """
    Aplica la poda Apriori para eliminar candidatos cuyos subconjuntos no son frecuentes.
    
    Parámetros:
    - candidates (list of tuples): Lista de candidatos k+1-itemsets.
    - frequent_itemsets_k (dict): Diccionario de itemsets frecuentes de tamaño k (clave: itemset, valor: soporte).

    Retorna:
    - valid_candidates (list of tuples): Lista de candidatos válidos (que pasan la poda).
    """
    valid_candidates = []  # Lista para almacenar los candidatos válidos
    
    # Iterar por cada candidato k+1-itemset
    for candidate in candidates:
        # Generar todos los subconjuntos de tamaño k
        subsets = list(combinations(candidate, len(candidate) - 1))
        
        # Verificar si TODOS los subconjuntos están en los itemsets frecuentes
        if all(subset in frequent_itemsets_k for subset in subsets):
            valid_candidates.append(candidate)  # Agregar candidato válido a la lista

    return valid_candidates


In [35]:
# Lista de 3-itemsets candidatos generados anteriormente
# (ejemplo: generados con combinations() a partir de productos frecuentes)
print(f"Número de candidatos iniciales (3-itemsets): {len(candidates_3itemsets)}")

Número de candidatos iniciales (3-itemsets): 364


In [36]:
for candidate in candidates_3itemsets:
    subsets = list(combinations(candidate, len(candidate) - 1))
    print(f"Subconjuntos de {candidate}: {subsets}")
    print(f"Frecuencia de subconjuntos: {[subset in frequent_item2_support for subset in subsets]}")


Subconjuntos de (26209, 47209, 47626): [(26209, 47209), (26209, 47626), (47209, 47626)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 45066): [(26209, 47209), (26209, 45066), (47209, 45066)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 28204): [(26209, 47209), (26209, 28204), (47209, 28204)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 39275): [(26209, 47209), (26209, 39275), (47209, 39275)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 21903): [(26209, 47209), (26209, 21903), (47209, 21903)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 21137): [(26209, 47209), (26209, 21137), (47209, 21137)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 24852): [(26209, 47209), (26209, 24852), (47209, 24852)]
Frecuencia de subconjuntos: [False, False, False]
Subconjuntos de (26209, 47209, 477

In [37]:
# Aplicar la poda Apriori para los 3-itemsets
valid_candidates_3itemsets = apriori_prune(candidates_3itemsets, frequent_item2_support)
print(f"Número de 3-itemsets válidos después de la poda: {len(valid_candidates_3itemsets)}")
print("Ejemplo de candidatos válidos:")
print(valid_candidates_3itemsets[:5])


Número de 3-itemsets válidos después de la poda: 0
Ejemplo de candidatos válidos:
[]


In [38]:
# Calcular soporte para los 3-itemsets válidos
frequent_item3_support = {}
for itemset in valid_candidates_3itemsets:
    support = calculate_support(itemset, transactions_dict.values())  # Usa la función que ya definiste
    if support >= min_support:  # Filtra los itemsets frecuentes
        frequent_item3_support[itemset] = support

print(f"Número de 3-itemsets frecuentes encontrados: {len(frequent_item3_support)}")


Número de 3-itemsets frecuentes encontrados: 0


In [39]:
# Crear DataFrame con los 3-itemsets frecuentes y su soporte
frequent_item3_df = pd.DataFrame({
    'itemset': list(frequent_item3_support.keys()),
    'support': list(frequent_item3_support.values())
})

print("DataFrame de 3-itemsets frecuentes:")
print(frequent_item3_df.head())


DataFrame de 3-itemsets frecuentes:
Empty DataFrame
Columns: [itemset, support]
Index: []
